### Importing Libraries

In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
from pprint import pprint
import psycopg2
import json

### Loading Data

In [2]:
yanki_df = pd.read_csv(r'dataset\raw_data\yanki_ecommerce.csv')

yanki_df.head()

,Order_ID,Customer_ID,Customer_Name,Product_ID,Product_Name,Brand,Category,Price,Quantity,Total_Price,Order_Date,Shipping_Address,City,State,Country,Postal_Code,Email,Phone_Number,Payment_Method,Transaction_Status
0,NaN,NaN,NaN,64b689a8-bf03-47d2-a5cc-0723baeb1606,major,Haynes PLC,perfume,52.34,2,104.68,2020-05-27 14:20:38.292975,"1648 Brown Bridge Apt. 846\nCaitlinland, MI 57992",Davidstad,Utah,China,50247,karen64@example.com,001-217-511-0290x8262,Credit Card,completed
1,NaN,NaN,NaN,64b689a8-bf03-47d2-a5cc-0723baeb1606,major,Haynes PLC,perfume,52.34,2,104.68,2020-05-27 14:20:38.292975,"1648 Brown Bridge Apt. 846\nCaitlinland, MI 57992",Davidstad,Utah,China,50247,karen64@example.com,001-217-511-0290x8262,Credit Card,completed
2,e32af09a-b6ab-497e-af3a-331e4d4ed6e7,e0d6cb3c-c4b0-4cfe-8225-b65d094d2424,Dominic Buchanan,2ef6e8fa-6a36-4515-b1c2-a0a700abf386,despite,"Lawson, Stone and Campos",perfume oil,250.57,2,501.14,2020-03-06 06:49:07.272117,"PSC 2224, Box 2284\nAPO AP 65880",Jordanborough,Arkansas,Kuwait,27879,margaret97@example.com,259.603.6134,Debit Card,completed
3,86eb8859-14ab-4d4e-9267-c5826f4e0c8e,fa3ca35a-5540-404b-a7eb-9001cdcbd840,Daniel Allen,3ba38e01-f8e7-4af2-9246-87ef0961d4f5,sea,Washington Group,perfume oil,179.81,6,1078.86,2020-07-31 07:09:30.231510,"83909 Johnson Mall\nTranberg, MS 99017",Lake Ginatown,Georgia,Saint Vincent and the Grenadines,39336,angela55@example.com,+1-869-659-4272x982,Credit Card,completed
4,7379b560-8897-4623-92a2-523ddcdc43a1,7ad4de53-e6d7-4cd3-99b8-13fb70fe7a34,Daniel Schmidt,a58c53bd-a34b-4541-b926-bec9eb84cac2,suddenly,Rodgers Ltd,perfume oil,600.55,7,4203.85,2021-05-11 04:33:09.707443,"82101 Johnson Flat\nWest Ryan, MS 29075",Sarahville,Ohio,France,30845,wayne59@example.org,292.840.0975x724,PayPal,pending


In [3]:
yanki_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1020 entries, 0 to 1019
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Order_ID            1000 non-null   object 
 1   Customer_ID         1000 non-null   object 
 2   Customer_Name       1000 non-null   object 
 3   Product_ID          1020 non-null   object 
 4   Product_Name        1020 non-null   object 
 5   Brand               1020 non-null   object 
 6   Category            1020 non-null   object 
 7   Price               1020 non-null   float64
 8   Quantity            1020 non-null   int64  
 9   Total_Price         1020 non-null   float64
 10  Order_Date          1020 non-null   object 
 11  Shipping_Address    1020 non-null   object 
 12  City                1020 non-null   object 
 13  State               1020 non-null   object 
 14  Country             1020 non-null   object 
 15  Postal_Code         1020 non-null   int64  
 16  Email 

In [4]:
yanki_df.columns

Index(['Order_ID', 'Customer_ID', 'Customer_Name', 'Product_ID',
       'Product_Name', 'Brand', 'Category', 'Price', 'Quantity', 'Total_Price',
       'Order_Date', 'Shipping_Address', 'City', 'State', 'Country',
       'Postal_Code', 'Email', 'Phone_Number', 'Payment_Method',
       'Transaction_Status'],
      dtype='object')

### Data Cleaning and Transformation

In [15]:
# Drop missing values
yanki_df.dropna(subset = ['Order_ID', 'Customer_ID'], inplace = True)

yanki_df

,Order_ID,Customer_ID,Customer_Name,Product_ID,Product_Name,Brand,Category,Price,Quantity,Total_Price,Order_Date,Shipping_Address,City,State,Country,Postal_Code,Email,Phone_Number,Payment_Method,Transaction_Status
2,e32af09a-b6ab-497e-af3a-331e4d4ed6e7,e0d6cb3c-c4b0-4cfe-8225-b65d094d2424,Dominic Buchanan,2ef6e8fa-6a36-4515-b1c2-a0a700abf386,despite,"Lawson, Stone and Campos",perfume oil,250.57,2,501.14,2020-03-06 06:49:07.272117,"PSC 2224, Box 2284\nAPO AP 65880",Jordanborough,Arkansas,Kuwait,27879,margaret97@example.com,259.603.6134,Debit Card,completed
3,86eb8859-14ab-4d4e-9267-c5826f4e0c8e,fa3ca35a-5540-404b-a7eb-9001cdcbd840,Daniel Allen,3ba38e01-f8e7-4af2-9246-87ef0961d4f5,sea,Washington Group,perfume oil,179.81,6,1078.86,2020-07-31 07:09:30.231510,"83909 Johnson Mall\nTranberg, MS 99017",Lake Ginatown,Georgia,Saint Vincent and the Grenadines,39336,angela55@example.com,+1-869-659-4272x982,Credit Card,completed
4,7379b560-8897-4623-92a2-523ddcdc43a1,7ad4de53-e6d7-4cd3-99b8-13fb70fe7a34,Daniel Schmidt,a58c53bd-a34b-4541-b926-bec9eb84cac2,suddenly,Rodgers Ltd,perfume oil,600.55,7,4203.85,2021-05-11 04:33:09.707443,"82101 Johnson Flat\nWest Ryan, MS 29075",Sarahville,Ohio,France,30845,wayne59@example.org,292.840.0975x724,PayPal,pending
5,30c57f86-2fed-43c0-a399-654c2c3bb18d,4b9b409c-19f2-41c0-bc7c-6556e0647ebb,John Gonzalez,e6021be5-90be-4199-b6c6-82542fb2973c,site,"Wilson, Scott and Johnson",perfume oil,414.36,9,3729.24,2023-09-12 01:47:56.290477,"4542 Gary Lane\nSouth Alexander, VI 27671",Lake Michelle,Arkansas,Sao Tome and Principe,30955,qsherman@example.com,+1-643-561-3912x262,PayPal,completed
6,41c29d81-9622-43c1-b537-7cdb320d3e6a,81b513ad-5c02-48cf-bff0-39e1440c4d22,Amber Benitez,60127671-ab32-4fb5-bfeb-1a7782ad835e,act,"Riddle, Alvarez and Robinson",perfume,704.05,9,6336.45,2021-08-20 18:02:04.254156,"6794 Caroline Crescent\nEast Karlside, GU 81379",Olsenville,Mississippi,Croatia,45429,derek85@example.com,3424148376,PayPal,pending
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,bb17468a-7648-4974-b3d6-9f9f1a78cb39,1e75eb3a-9ba5-4ea6-82ec-886abe298058,Jeremy Farrell,ad5204d7-e5c2-417c-91f7-5fa07619ef54,officer,Thornton Group,perfume,118.46,3,355.38,2023-08-07 12:34:10.009932,"440 Carson Port Suite 634\nLake Heather, MA 22883",West Danielle,South Carolina,Iran,17863,leejohn@example.com,922.207.8157x85481,PayPal,pending
1016,184827be-c3bd-40a1-a2bb-8b085d58ddc0,c53c32ef-2e64-4ac4-ac48-3707ab1ebd1f,Marc Hudson,0ea15074-b823-4345-924c-aeb9fafe9571,treat,Guzman-Petersen,perfume,91.14,2,182.28,2024-01-14 09:27:21.372714,"75333 Clayton Forges Apt. 292\nLake Matthew, D...",Whitemouth,Arkansas,Japan,24756,james85@example.org,908.901.4896x12666,PayPal,completed
1017,2dac83f2-fddb-4761-b211-ea716feee91c,ebc19a03-a32d-4d3c-9976-f30cee3d3a70,Devin Joseph,e911b0a9-75fb-4c17-b552-52be0d19f856,watch,Jones-May,perfume,246.73,10,2467.30,2021-01-23 10:25:25.233434,"426 Amy Islands\nCollinsberg, MT 21928",South Stephanie,Indiana,Timor-Leste,54502,chadbenson@example.com,(718)789-6289,PayPal,pending
1018,08341169-2d82-41d4-8273-fda624f99042,0598403f-7a42-44a9-994e-9f23feb20394,Spencer Elliott,9c9792f9-4dc2-43b2-98f4-e989cf1bdb3a,toward,Richardson PLC,perfume,647.26,10,6472.60,2022-04-15 23:15:41.204791,"01490 Lopez Garden\nPort Adamton, ND 35200",Thompsonfort,South Dakota,Pakistan,78892,eric98@example.org,366-506-0469x728,PayPal,pending


In [6]:
yanki_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 2 to 1019
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Order_ID            1000 non-null   object 
 1   Customer_ID         1000 non-null   object 
 2   Customer_Name       1000 non-null   object 
 3   Product_ID          1000 non-null   object 
 4   Product_Name        1000 non-null   object 
 5   Brand               1000 non-null   object 
 6   Category            1000 non-null   object 
 7   Price               1000 non-null   float64
 8   Quantity            1000 non-null   int64  
 9   Total_Price         1000 non-null   float64
 10  Order_Date          1000 non-null   object 
 11  Shipping_Address    1000 non-null   object 
 12  City                1000 non-null   object 
 13  State               1000 non-null   object 
 14  Country             1000 non-null   object 
 15  Postal_Code         1000 non-null   int64  
 16  Email      

In [ ]:
# Convert Order_date from string to datetime object
yanki_df['Order_Date'] = pd.to_datetime(yanki_df['Order_Date'])

In [8]:
yanki_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 2 to 1019
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Order_ID            1000 non-null   object        
 1   Customer_ID         1000 non-null   object        
 2   Customer_Name       1000 non-null   object        
 3   Product_ID          1000 non-null   object        
 4   Product_Name        1000 non-null   object        
 5   Brand               1000 non-null   object        
 6   Category            1000 non-null   object        
 7   Price               1000 non-null   float64       
 8   Quantity            1000 non-null   int64         
 9   Total_Price         1000 non-null   float64       
 10  Order_Date          1000 non-null   datetime64[ns]
 11  Shipping_Address    1000 non-null   object        
 12  City                1000 non-null   object        
 13  State               1000 non-null   object        
 1

In [16]:
# Customer Table 
Customer_Table = yanki_df[['Customer_ID', 'Customer_Name', 'Email', 'Phone_Number']].copy().drop_duplicates().reset_index(drop = True)
Customer_Table.head()

,Customer_ID,Customer_Name,Email,Phone_Number
0,e0d6cb3c-c4b0-4cfe-8225-b65d094d2424,Dominic Buchanan,margaret97@example.com,259.603.6134
1,fa3ca35a-5540-404b-a7eb-9001cdcbd840,Daniel Allen,angela55@example.com,+1-869-659-4272x982
2,7ad4de53-e6d7-4cd3-99b8-13fb70fe7a34,Daniel Schmidt,wayne59@example.org,292.840.0975x724
3,4b9b409c-19f2-41c0-bc7c-6556e0647ebb,John Gonzalez,qsherman@example.com,+1-643-561-3912x262
4,81b513ad-5c02-48cf-bff0-39e1440c4d22,Amber Benitez,derek85@example.com,3424148376


In [17]:
# Product Table
Product_Table = yanki_df[['Product_ID', 'Customer_ID', 'Product_Name', 'Brand', 'Category', 'Price']].copy().drop_duplicates().reset_index(drop = True)
Product_Table.head()

,Product_ID,Customer_ID,Product_Name,Brand,Category,Price
0,2ef6e8fa-6a36-4515-b1c2-a0a700abf386,e0d6cb3c-c4b0-4cfe-8225-b65d094d2424,despite,"Lawson, Stone and Campos",perfume oil,250.57
1,3ba38e01-f8e7-4af2-9246-87ef0961d4f5,fa3ca35a-5540-404b-a7eb-9001cdcbd840,sea,Washington Group,perfume oil,179.81
2,a58c53bd-a34b-4541-b926-bec9eb84cac2,7ad4de53-e6d7-4cd3-99b8-13fb70fe7a34,suddenly,Rodgers Ltd,perfume oil,600.55
3,e6021be5-90be-4199-b6c6-82542fb2973c,4b9b409c-19f2-41c0-bc7c-6556e0647ebb,site,"Wilson, Scott and Johnson",perfume oil,414.36
4,60127671-ab32-4fb5-bfeb-1a7782ad835e,81b513ad-5c02-48cf-bff0-39e1440c4d22,act,"Riddle, Alvarez and Robinson",perfume,704.05


In [26]:
# Shipping Table
Shipping_Table = yanki_df[['Product_ID', 'Shipping_Address', 'City', 'State', 'Country', 'Postal_Code']].copy().drop_duplicates().reset_index(drop = True)


# Assign a unique primary key for tax_id using a dedicated numbering system
Shipping_Table['Shipping_ID'] = range(5, len(Shipping_Table) + 5)  # Sequential numbering starting at 5

Shipping_Table = Shipping_Table[['Shipping_ID', 'Product_ID', 'Shipping_Address', 'City', 'State', 'Country', 'Postal_Code']].copy().drop_duplicates().reset_index(drop = True)


Shipping_Table.head()

,Shipping_ID,Product_ID,Shipping_Address,City,State,Country,Postal_Code
0,5,2ef6e8fa-6a36-4515-b1c2-a0a700abf386,"PSC 2224, Box 2284\nAPO AP 65880",Jordanborough,Arkansas,Kuwait,27879
1,6,3ba38e01-f8e7-4af2-9246-87ef0961d4f5,"83909 Johnson Mall\nTranberg, MS 99017",Lake Ginatown,Georgia,Saint Vincent and the Grenadines,39336
2,7,a58c53bd-a34b-4541-b926-bec9eb84cac2,"82101 Johnson Flat\nWest Ryan, MS 29075",Sarahville,Ohio,France,30845
3,8,e6021be5-90be-4199-b6c6-82542fb2973c,"4542 Gary Lane\nSouth Alexander, VI 27671",Lake Michelle,Arkansas,Sao Tome and Principe,30955
4,9,60127671-ab32-4fb5-bfeb-1a7782ad835e,"6794 Caroline Crescent\nEast Karlside, GU 81379",Olsenville,Mississippi,Croatia,45429


In [19]:
# Order Table
Order_Table = yanki_df[['Order_ID', 'Product_ID', 'Quantity', 'Total_Price', 'Order_Date']].copy().drop_duplicates().reset_index(drop = True)
Order_Table.head()

,Order_ID,Product_ID,Quantity,Total_Price,Order_Date
0,e32af09a-b6ab-497e-af3a-331e4d4ed6e7,2ef6e8fa-6a36-4515-b1c2-a0a700abf386,2,501.14,2020-03-06 06:49:07.272117
1,86eb8859-14ab-4d4e-9267-c5826f4e0c8e,3ba38e01-f8e7-4af2-9246-87ef0961d4f5,6,1078.86,2020-07-31 07:09:30.231510
2,7379b560-8897-4623-92a2-523ddcdc43a1,a58c53bd-a34b-4541-b926-bec9eb84cac2,7,4203.85,2021-05-11 04:33:09.707443
3,30c57f86-2fed-43c0-a399-654c2c3bb18d,e6021be5-90be-4199-b6c6-82542fb2973c,9,3729.24,2023-09-12 01:47:56.290477
4,41c29d81-9622-43c1-b537-7cdb320d3e6a,60127671-ab32-4fb5-bfeb-1a7782ad835e,9,6336.45,2021-08-20 18:02:04.254156


In [27]:
# Payment Table
Payment_Table = yanki_df[['Order_ID', 'Payment_Method', 'Transaction_Status']].copy().drop_duplicates().reset_index(drop = True)


# Assign a unique primary key for tax_id using a dedicated numbering system
Payment_Table['Payment_ID'] = range(99, len(Payment_Table) + 99)  # Sequential numbering starting at 99

Payment_Table = Payment_Table[['Payment_ID', 'Order_ID', 'Payment_Method', 'Transaction_Status']].copy().drop_duplicates().reset_index(drop = True)


Payment_Table.head()

,Payment_ID,Order_ID,Payment_Method,Transaction_Status
0,99,e32af09a-b6ab-497e-af3a-331e4d4ed6e7,Debit Card,completed
1,100,86eb8859-14ab-4d4e-9267-c5826f4e0c8e,Credit Card,completed
2,101,7379b560-8897-4623-92a2-523ddcdc43a1,PayPal,pending
3,102,30c57f86-2fed-43c0-a399-654c2c3bb18d,PayPal,completed
4,103,41c29d81-9622-43c1-b537-7cdb320d3e6a,PayPal,pending


In [28]:
# Save the Tables to CSVs
Customer_Table.to_csv(r'dataset\clean_data\Customer_Table.csv', index = False)
Product_Table.to_csv(r'dataset\clean_data\Product_Table.csv', index = False)
Shipping_Table.to_csv(r'dataset\clean_data\Shipping_Table.csv', index = False)
Order_Table.to_csv(r'dataset\clean_data\Order_Table.csv', index = False)
Payment_Table.to_csv(r'dataset\clean_data\Payment_Table.csv', index = False)

### Loading Layer

In [87]:
# Load environment variables from .env file
load_dotenv()

# Develop a function to get the Database connections
def get_db_connection():
    connection = psycopg2.connect(
        # Retrieve environment variables
        host = os.getenv('HOST'),
        database = os.getenv('DATABASE'),
        user = os.getenv('USER'),
        password = os.getenv('PASSWORD')
    )

    return connection
# Connect to the Database
conn = get_db_connection()


In [88]:
# Create a function that sets up the schema and tables
def create_tables():
    conn = get_db_connection()
    cursor = conn.cursor()
    create_table_query = '''
                            CREATE SCHEMA IF NOT EXISTS Yanki_Ecommerce;
                            
                            DROP TABLE IF EXISTS Yanki_Ecommerce.Customer_Table CASCADE;
                            DROP TABLE IF EXISTS Yanki_Ecommerce.Product_Table CASCADE;
                            DROP TABLE IF EXISTS Yanki_Ecommerce.Shipping_Table CASCADE;
                            DROP TABLE IF EXISTS Yanki_Ecommerce.Order_Table CASCADE;
                            DROP TABLE IF EXISTS Yanki_Ecommerce.Payment_Table CASCADE;

                            CREATE TABLE Yanki_Ecommerce.Customer_Table (
                                Customer_ID UUID PRIMARY KEY,
                                Customer_Name VARCHAR(100000),
                                Email VARCHAR(100000),
                                Phone_Number VARCHAR(100000)
                            );



                            CREATE TABLE Yanki_Ecommerce.Product_Table (
                                Product_ID UUID PRIMARY KEY,
                                Customer_ID UUID,
                                Product_Name VARCHAR(100000),
                                Brand VARCHAR(100000),
                                Category VARCHAR(100000),
                                Price FLOAT,
                                FOREIGN KEY (Customer_ID) REFERENCES Yanki_Ecommerce.Customer_Table(Customer_ID)
                            );



                             CREATE TABLE Yanki_Ecommerce.Shipping_Table (
                                Shipping_ID INTEGER PRIMARY KEY,
                                Product_ID UUID,
                                Shipping_Address VARCHAR(100000),
                                City VARCHAR(100000),
                                State VARCHAR(100000),
                                Country VARCHAR(100000),
                                Postal_Code INTEGER,
                                FOREIGN KEY (Product_ID) REFERENCES Yanki_Ecommerce.Product_Table(Product_ID)
                            );



                            CREATE TABLE Yanki_Ecommerce.Order_Table (
                                Order_ID UUID PRIMARY KEY,
                                Product_ID UUID,
                                Quantity INTEGER,
                                Total_Price FLOAT,
                                Order_Date DATE,
                                FOREIGN KEY (Product_ID) REFERENCES Yanki_Ecommerce.Product_Table(Product_ID)
                            );



                             CREATE TABLE Yanki_Ecommerce.Payment_Table (
                                Payment_ID INTEGER PRIMARY KEY,
                                Order_ID UUID,
                                Payment_Method VARCHAR(100000),
                                Transaction_Status VARCHAR(100000),
                                FOREIGN KEY (Order_ID) REFERENCES Yanki_Ecommerce.Order_Table(Order_ID) 
                            );
                            '''
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()
create_tables()

In [89]:
# import csv
# def load_data_from_csv(csv_path):
#     conn = get_db_connection()
#     cursor = conn.cursor()
#     with open(csv_path, 'r') as file:
#         reader = csv.reader(file)
#         next(reader)
#         for row in reader:
#             cursor.execute('''
#                             INSERT INTO Yanki_Ecommerce.Customer_Table (Customer_ID, Customer_Name, Email, Phone_Number)
#                             VALUES (%s, %s, %s, %s);''',
#                             row
#                         )

#     conn.commit()
#     cursor.close()
#     conn.close()

# csv_file_path = r'dataset\clean_data\Customer_Table.csv'    

# load_data_from_csv(csv_file_path)

In [90]:
import csv

# Function to load data from a CSV file into the database table
def load_data_from_csv(csv_path, query):
    """
    This function reads data from a CSV file and inserts it into the specified database table.

    Parameters:
    csv_path (str): The path to the CSV file containing data.
    query (str): The SQL INSERT query for the respective table.
    """
    conn = get_db_connection()
    cursor = conn.cursor()
    with open(csv_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            cursor.execute(query, row)

    conn.commit()
    cursor.close()
    conn.close()              


# SQL query to insert data into the respective tables
Customer_Table_query = '''
    INSERT INTO Yanki_Ecommerce.Customer_Table (Customer_ID, Customer_Name, Email, Phone_Number)
    VALUES (%s, %s, %s, %s);
'''
            
Product_Table_query = '''
    INSERT INTO Yanki_Ecommerce.Product_Table (Product_ID, Customer_ID, Product_Name, Brand, Category, Price)
    VALUES (%s, %s, %s, %s, %s, %s);
'''
                        

Shipping_Table_query = '''
    INSERT INTO Yanki_Ecommerce.Shipping_Table (Shipping_ID, Product_ID, Shipping_Address, City, State, Country, Postal_Code)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
'''
    

Order_Table_query = '''
    INSERT INTO Yanki_Ecommerce.Order_Table (Order_ID, Product_ID, Quantity, Total_Price, Order_Date)
    VALUES (%s, %s, %s, %s, %s);
'''


Payment_Table_query = '''
    INSERT INTO Yanki_Ecommerce.Payment_Table (Payment_ID, Order_ID, Payment_Method, Transaction_Status)
    VALUES (%s, %s, %s, %s);
'''

# Paths to the respective CSV files containing the data for each table
Customer_Table_csv_path = r'dataset\clean_data\Customer_Table.csv'    
Product_Table_csv_path = r'dataset\clean_data\Product_Table.csv'   
Shipping_Table_csv_path = r'dataset\clean_data\Shipping_Table.csv'
Order_Table_csv_path = r'dataset/clean_data/Order_Table.csv'
Payment_Table_csv_path = r'dataset/clean_data/Payment_Table.csv'


# Load data into the respective tables
load_data_from_csv(Customer_Table_csv_path, Customer_Table_query)
load_data_from_csv(Product_Table_csv_path, Product_Table_query)
load_data_from_csv(Shipping_Table_csv_path, Shipping_Table_query)
load_data_from_csv(Order_Table_csv_path, Order_Table_query)
load_data_from_csv(Payment_Table_csv_path, Payment_Table_query)


In [91]:
display(Shipping_Table.columns)

Index(['Shipping_ID', 'Product_ID', 'Shipping_Address', 'City', 'State',
       'Country', 'Postal_Code'],
      dtype='object')

In [92]:
display(Customer_Table.info())
display(Customer_Table.columns)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 990 entries, 0 to 989
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Customer_ID    990 non-null    object
 1   Customer_Name  990 non-null    object
 2   Email          990 non-null    object
 3   Phone_Number   990 non-null    object
dtypes: object(4)
memory usage: 31.1+ KB


None

Index(['Customer_ID', 'Customer_Name', 'Email', 'Phone_Number'], dtype='object')

In [98]:
Shipping_Table.head()

,Shipping_ID,Product_ID,Shipping_Address,City,State,Country,Postal_Code
0,5,2ef6e8fa-6a36-4515-b1c2-a0a700abf386,"PSC 2224, Box 2284\nAPO AP 65880",Jordanborough,Arkansas,Kuwait,27879
1,6,3ba38e01-f8e7-4af2-9246-87ef0961d4f5,"83909 Johnson Mall\nTranberg, MS 99017",Lake Ginatown,Georgia,Saint Vincent and the Grenadines,39336
2,7,a58c53bd-a34b-4541-b926-bec9eb84cac2,"82101 Johnson Flat\nWest Ryan, MS 29075",Sarahville,Ohio,France,30845
3,8,e6021be5-90be-4199-b6c6-82542fb2973c,"4542 Gary Lane\nSouth Alexander, VI 27671",Lake Michelle,Arkansas,Sao Tome and Principe,30955
4,9,60127671-ab32-4fb5-bfeb-1a7782ad835e,"6794 Caroline Crescent\nEast Karlside, GU 81379",Olsenville,Mississippi,Croatia,45429
